In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client, LocalCluster

client = Client() # Note that `memory_limit` is the limit **per worker**.
# n_workers=4,
#                 threads_per_worker=1,
#                 memory_limit='3GB'
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [ ]:
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import scipy.constants

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

In [ ]:
matplotlib.rcParams.update({'font.size': 16})

### Open and resave file

In [ ]:
# file path to data and configs
#prefix = "/Users/abroome/Documents/SDR/uhd_radar/data/20230710_095333"
#prefix = "/Users/abroome/Desktop/McMurdo2022/11222022_anna/20221122_135614"
#prefix = "/home/radioglaciolgy/anna/uhd_radar/data/20230719_103400"

#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231101_eyas_loopback_cal/20231101_200430" # Eyas 1 (Green), w/ LNA and filters
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231101_eyas_loopback_cal/20231101_210720" # Eyas 1 (Green), NO LNA, w/ filters
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231101_eyas_loopback_cal/20231101_211515" # Eyas 1 (Green), NO LNA, NO FILTERS
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231101_eyas_loopback_cal/20231101_202028" # Eyas 2 (Black), w/ LNA and filters
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231101_eyas_loopback_cal/20231101_203306" # Eyas 3 (Yellow), w/ LNA and filters
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231101_eyas_loopback_cal/20231101_204529" # Eyas 4 (Orange), w/ LNA and filters
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231101_eyas_loopback_cal/20231101_194641" # Eyas 5 (Orange, Non-Field), No LNA or Filters
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231101_eyas_loopback_cal/20231101_212724" # Eyas 5 (Orange, Non-Field), No LNA or Filters, Repeat

#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/drone/radar_data/orca_paper_data_files/phase_noise/b205/20240222_203345"

#prefix = "/Volumes/T7 Shield/Summit2023/07252023_anna/20230725_161051" # Anna field spectrogram
#prefix = "/Volumes/Extreme SSD/orca_paper_data_files/spectrogram/b205/20240226_195954" # Thomas spectrogram, no phase dithering
#prefix = "/Volumes/Extreme SSD/orca_paper_data_files/spectrogram/b205/20240226_194526" # Thomas spectrogram, phase dithering
#prefix = "/Volumes/Extreme SSD/orca_paper/20240226_105916" # Anna loopback spectrogram, phase dithering
prefix = "/Volumes/Extreme SSD/orca_paper/20240226_105437" # Anna loopback spectrogram, no phase dithering
prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/drone/radar_data/20230723-summit-day3-2start/20230723_104248"

# resave data as zarr for dask processing
zarr_base_location="/home/thomas/Documents/StanfordGrad/RadioGlaciology/test_tmp_zarr_cache/"
zarr_path = pr.save_radar_data_to_zarr(prefix, zarr_base_location=zarr_base_location)

# open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1000})

### Enter processing parameters

In [ ]:
#zero_sample_idx = 36 # X310, fs = 20 MHz
#zero_sample_idx = 63 # X310, fs = 50 MHz
zero_sample_idx = 159 # B205mini, fs = 56 MHz
#zero_sample_idx = 166 # B205mini, fs = 20 MHz

nstack = 1 # number of pulses to stack

modify_rx_window = False # set to true if you want to window the reference chirp only on receive, false uses ref chirp as transmitted in config file
rx_window = "rectangular" # what you want to change the rx window to if modify_rx_window is true

#dielectric_constant = 3.17 # ice (air = 1, 66% velocity coax = 2.2957)
dielectric_constant = 2.2957 # COAX (air = 1, 66% velocity coax = 2.2957)
sig_speed = scipy.constants.c / np.sqrt(dielectric_constant)

### Generate reference chirp

In [ ]:
if modify_rx_window:
    config = copy.deepcopy(raw.config)
    config['GENERATE']['window'] = rx_window
else:
    config = raw.config

chirp_ts, ref_chirp = generate_chirp(config)

### View raw pulse in time domain to check for clipping

In [ ]:
single_pulse_raw = raw.radar_data[{'pulse_idx': 100}].compute()
plot1 = np.real(single_pulse_raw).hvplot.line(x='fast_time', color='red') * np.imag(single_pulse_raw).hvplot.line(x='fast_time')

plot1 = plot1.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
plot1

### Clean and stack data

In [ ]:
stacked = pr.fill_errors(raw, error_fill_value=0.0) # fill receiver errors with 0s

stacked = pr.stack(stacked, nstack) # stack 

### Pulse compress data

In [ ]:
compressed = pr.pulse_compress(stacked, ref_chirp,
                               fs=stacked.config['GENERATE']['sample_rate'],
                               zero_sample_idx=zero_sample_idx,
                               signal_speed=sig_speed)

compressed_power = xr.apply_ufunc(
    lambda x: 20*np.log10(np.abs(x)),
    compressed,
    dask="parallelized"
)

### View 1D pulse compressed data

In [ ]:
plot1D = compressed_power.radar_data[0,:].hvplot.line()
plot1D = plot1D * compressed_power.radar_data[-1,:].hvplot.line()
# relevant options: xlim(-80,1000)

plot1D = plot1D.opts(xlabel='Reflection Distance (m)', ylabel='Return Power (dB)')
plot1D.opts(xlim=(-50,200), ylim=(-120, -40), show_grid=True)

### View 2D pulse compressed data (radargram)

In [ ]:
# USING HOLOVIEWS (sometimes breaks)
plot2D = compressed_power.swap_dims({'pulse_idx': 'slow_time', 'travel_time': 'reflection_distance'}).hvplot.quadmesh(x='slow_time', cmap='inferno', flip_yaxis=True)
# relevant options: ylim=(100,-50), clim=(-90,-40)

plot2D.opts(xlabel='Slow Time (s)', ylabel='Depth (m)', clabel='Return Power (dB)')
plot2D.opts(ylim=(-10, 70), clim=(-120, -40))

In [ ]:
# USING MATPLOTLIB (sometimes takes a while)
fig, ax = plt.subplots(1,1, figsize=(10,6), facecolor='white')

p = ax.pcolormesh(compressed_power.slow_time, compressed_power.reflection_distance, compressed_power.radar_data.transpose(), shading='auto', cmap='inferno')
ax.invert_yaxis()
clb = fig.colorbar(p, ax=ax)
clb.set_label('Return Power (dB)')
ax.set_xlabel('Slow Time (s)')
ax.set_ylabel('Distance to Reflector (m)')
# relevant options: ax.set_ylim=(100,-50), ax.set_xlim=(0, 1), vmin=-90, vmax=40
ax.set_ylim(100, -50)

### View spectrogram of stacked data

In [ ]:
inpt = raw
inpt["radar_data"].shape

In [ ]:
num_presums = raw.attrs["config"]["CHIRP"]["num_presums"]

In [ ]:
# data = stacked["radar_data"].to_numpy()
n = 1000000
normalize = True

pulse = pr.stack(inpt, n)[{'pulse_idx':0}]["radar_data"].to_numpy()

f, t, S = scipy.signal.spectrogram(
    pulse,
    fs=raw.attrs["config"]["GENERATE"]["sample_rate"],
    window='flattop',
    nperseg=128,
    noverlap=64,
    scaling='density', mode='psd',
    return_onesided=False
)

if normalize:
    S /= np.max(S)

In [ ]:
fig, ax = plt.subplots(facecolor='white', figsize=(10,6))
freq_mhz = (np.fft.fftshift(f) + raw.attrs['config']['RF0']['freq']) / 1e6
pcm = ax.pcolormesh(t, freq_mhz, 10*np.log10(np.abs(np.fft.fftshift(S, axes=0))), shading='nearest') #  vmin=-420, vmax=-200
clb = fig.colorbar(pcm, ax=ax)
clb.set_label('Power [dB]')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Frequency [MHz]')
#ax.set_title(f"Spectrogram of received data with n_stack={n}")
ax.text(0, 1.05, prefix.split("/")[-1] + "\n" + f"n_stack * num_presums = {n * num_presums}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes, fontdict={'size': 12})
fig.tight_layout()
plt.show()

In [ ]:
fig.savefig(f"orca_paper/outputs/{raw.basename}_ft_spectrogram_n{n}.png", dpi=300)

### View Power Spectrum of All Received Data

In [ ]:
single_stack = pr.stack(raw, raw.radar_data.shape[1])

data_rx_fft = da.fft.fft(raw.radar_data, axis=0) / raw.radar_data.shape[0]
stacked_fft = da.fft.fft(stacked.radar_data, axis=0) / stacked.radar_data.shape[0]
full_fft = da.fft.fft(single_stack.radar_data, axis=0) / single_stack.radar_data.shape[0]

data_rx_fft_pwr = 20*da.log10(da.abs(data_rx_fft))
stacked_fft_pwr = 20*da.log10(da.abs(stacked_fft))
full_fft_pwr = 20*da.log10(da.abs(full_fft))

#data_rx_fft_pwr.shape

In [ ]:
# fig, axs = plt.subplots(2,1)
fig, axs = plt.subplots(facecolor='white', figsize=(10,6))
freqs = np.fft.fftshift(np.fft.fftfreq(data_rx_fft_pwr.shape[0], d=1/raw.config['GENERATE']['sample_rate']))
axs.plot(freqs/1e6, np.fft.fftshift(data_rx_fft_pwr[:,0]), label='Single Pulse')
axs.plot(freqs/1e6, np.fft.fftshift(stacked_fft_pwr[:,0]), label='Single Stack')
axs.plot(freqs/1e6, np.fft.fftshift(full_fft_pwr[:,0]), label='Full File')
axs.set_xlabel('Frequency [MHz]')
axs.set_ylabel('Power [dB]')
axs.set_title('Spectrum -- Power')
axs.grid()
axs.legend()

# axs[1].plot(freqs/1e6, np.fft.fftshift(np.angle(data_rx_fft[:,0])))
# axs[1].plot(freqs/1e6, np.fft.fftshift(np.angle(stacked_fft[:,0])))
# axs[1].plot(freqs/1e6, np.fft.fftshift(np.angle(full_fft[:,0])))
# axs[1].set_xlabel('Frequency [MHz]')
# axs[1].set_ylabel('Phase [rad]')
# axs[1].set_title('Spectrum -- Phase')
# axs[1].grid()
# fig.tight_layout()